In [8]:
from pyspark.sql import SparkSession

spark = SparkSession \
  .builder \
  .appName("PySpark pipeline") \
  .getOrCreate()

In [9]:
data = spark.read.csv("data/housing.csv", header=True, inferSchema=True)

data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [10]:
filtered_data = data.na.drop(subset=['total_bedrooms'])
filtered_data.count()

20433

In [11]:
train_data, test_data = filtered_data.randomSplit([0.8, 0.2], seed=42)
print("Train size: ", train_data.count())
print("Test size: ", test_data.count())

Train size:  16395
Test size:  4038


In [12]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler

indexer = StringIndexer(inputCol='ocean_proximity', outputCol='ocean_proximity_index')
encoder = OneHotEncoder(inputCol='ocean_proximity_index', outputCol='ocean_proximity_vec', dropLast=False)


feature_cols = ['housing_median_age', 'total_rooms', 'total_bedrooms',
                'population', 'households', 'median_income', 'ocean_proximity_vec']
assembler = VectorAssembler(inputCols=feature_cols, outputCol='unscaled_features')
scaler = StandardScaler(inputCol='unscaled_features', outputCol='features', withMean=True, withStd=True)

lr = LinearRegression(featuresCol='features', labelCol='median_house_value', regParam=0.001)

pipeline = Pipeline(stages=[indexer, encoder, assembler, scaler, lr])

pipeline_model = pipeline.fit(train_data)

In [13]:
test_predictions = pipeline_model.transform(test_data)

In [14]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator_mae = RegressionEvaluator(labelCol='median_house_value', predictionCol='prediction', metricName='mae')
mae = evaluator_mae.evaluate(test_predictions)
print(f"Mean Absolute Error (MAE): {mae}")

Mean Absolute Error (MAE): 50597.33640580943
